In [ ]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: SkiaSharp"

#!import config/Settings.cs
#!import plugins/FilesPlugin.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TextToImage;

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010


var settings = Settings.LoadFromFile("config/settings.json");
var builder = Kernel.CreateBuilder();

builder
    .AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey)
    .AddAzureOpenAITextToImage("dall-e-3", settings.azureEndpoint, settings.apiKey)
    .Plugins.AddFromType<FilesPlugin>();


var kernel = builder.Build();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();
var dallE = kernel.GetRequiredService<ITextToImageService>();

In [ ]:
var systemMessage = 
"""
   You are an assistant that helps with organizing files. 
   Explicit user consent is required before proceeding with the actually file organization action.
   Do not include any special encoding in the file paths, just use the plain text file paths, no quotes.
   Don't tell the user that you are a bot. Just act like a helpful assistant that is helping with Desktop file organization tasks.
""";

var chat = new ChatHistory(systemMessage);

In [ ]:
var promptSettings = new OpenAIPromptExecutionSettings()
    {
        MaxTokens = 4000,
        ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions,
    };

Func<string, Task> Chat = async (string input) => {
    Console.WriteLine($"User:");
    Console.WriteLine($"{input}\n");
    chat.AddUserMessage(input);

     StringBuilder sb = new();
    await foreach (var message in chatGPT.GetStreamingChatMessageContentsAsync(chat, promptSettings, kernel))
    {
        sb.Append(message.Content);
    }
    var assistantReply = sb.ToString();
    chat.AddAssistantMessage(assistantReply);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
await Chat("List all files");

In [ ]:
await Chat("Please read and provide a summary of the poem. Kindly summarize the poem in two sentences.");

In [ ]:
#pragma warning disable SKEXP0001

var imageUrl = await dallE.GenerateImageAsync(chat.Last().Content, 1024, 1024);
await SkiaUtils.ShowImage(imageUrl, 1024, 1024);